# `DPRPy Assignment 2`
### By: Naveen Tiwari

## `Overview`
- This document is related to the `second assignment` from the subject `Data Processing in R and Python`. It Contains the codes for each queries, which includes compare the codes with the asked `SQLite query`, using the `.equals() method`. 

- This report includes solution of the SQlite query using Python codes based on `pandas and numpy library`. 
- I have imported my Python codes into the notebook, in order to do the comparision.
- The imported data from the `CSV files` has been loaded to the temporary `SQLite database` which will be used to run the reference `SQL solutions`.

### `Imprting the required libraries, Python solutions and CSV files:`

In [9]:
import numpy as np
import pandas as pd
from Naveen_Tiwari_assignment_2 import *


In [10]:
Posts = pd.read_csv("/Users/naveen/Downloads/Posts.csv", compression = None)
Users = pd.read_csv("/Users/naveen/Downloads/Users.csv", compression = None)
Badges = pd.read_csv("/Users/naveen/Downloads/Badges.csv", compression = None)
Comments = pd.read_csv("/Users/naveen/Downloads/Comments.csv", compression = None)
Votes = pd.read_csv("/Users/naveen/Downloads/Votes.csv", compression = None)

### `Creating SQLite database`:

In [11]:
import os, os.path
import sqlite3
import tempfile

# path to database file
db = os.path.join(tempfile.mkdtemp(), 'assig2.db')

if os.path.isfile(db): # if this file already exists...
    os.remove(db) # ...we will remove it

conn = sqlite3.connect(db) # create the connection

Badges.to_sql("Badges", conn) # import the data frame into the database Comments.to_sql("Comments", conn)
Comments.to_sql("Comments", conn)
Posts.to_sql("Posts", conn)
Users.to_sql("Users", conn)
Votes.to_sql("Votes", conn)

1002408

## `First Query`


In [12]:
# SQL
result_sql = pd.read_sql_query("""
SELECT STRFTIME('%Y', CreationDate) AS Year, COUNT(*) AS TotalNumber FROM Posts
GROUP BY Year
""", conn)

# Pandas
res = solution_1(Posts)

# Compare
result_sql.equals(res)


True

**Query explanation:**

- The `SQL query` asked to select just the `Year part` of the "CreationDate column" from the `Posts` csv file, and count the number of occurrence of each Year.

- So I just selected the Year part to get the appropriate date format, and renamed the  number of occurrence column as `TotalNumber`.

## `Second Query`


In [13]:
#SQL
result_sql = pd.read_sql_query("""
SELECT Id, DisplayName, SUM(ViewCount) AS TotalViews FROM Users
JOIN (
SELECT OwnerUserId, ViewCount FROM Posts WHERE PostTypeId = 1 ) AS Questions
ON Users.Id = Questions.OwnerUserId GROUP BY Id
ORDER BY TotalViews DESC
LIMIT 10
""", conn)

# Pandas
res = solution_2(Users, Posts)

# Compare
result_sql.equals(res.reset_index(drop=True))


True

**Query explanation:**

- In the SQL query it was asked to select some columns from the `Posts and Users` csv file and then, rename it and perform some operations.After that we have to `inner join` them. Then we have to order it by "TotalViews" column by `decreasing, with a limit of 10`.

- I selected the data only when `PosTtypeId = 1` from the "Posts csv file", and other columns as "Questions". Then I got the `sum of column ViewCount as TotalViews`.
After that performed it's `inner join` with of "Users csv file Id's part". Then we ordered the data of  `TotalViews` as decreasing with the `limit of 10`.



## `Third Query`

In [14]:
#SQL
result_sql= pd.read_sql_query("""
SELECT Year, Name, MAX((Count * 1.0) / CountTotal) AS MaxPercentage
FROM (
SELECT BadgesNames.Year, BadgesNames.Name, BadgesNames.Count, BadgesYearly.CountTotal
FROM (
SELECT Name, COUNT(*) AS Count, STRFTIME('%Y', Badges.Date) AS Year FROM Badges
GROUP BY Name, Year
) AS BadgesNames 
JOIN (
SELECT COUNT(*) AS CountTotal, STRFTIME('%Y', Badges.Date) AS Year FROM Badges
GROUP BY YEAR
) AS BadgesYearly
ON BadgesNames.Year = BadgesYearly.Year
)
GROUP BY Year
""", conn)

#pandas
res = solution_3(Badges) 

#compare
result_sql.equals(res)



True

**Query explanation:**

- The SQL query asked to compute the `maximum percentage` of badges per `year by names`. so that we can check number of badges assoiciated with a name, in comparision to the total number of badges per year.

- For `BadgesNames`, I selected the year part to get the required date format from `column Date of Badges csv file`.After that I did the counted the frequency of each year and did the renaming of the Date column as `Year` and the count part as `Count`. The same I did for `BadgesYearly` part, but here we just had to select count as `CountTotal` and `Year`.
- Then I did the `inner join` of `BadgesNames` and `BadgesYearly` by `Year`. After that I divided the Count and CountTotal column, and calculated the Max part, and after that did the renaming to get the desired results.


## `Fourth Query`

In [15]:
#SQL
result_sql= pd.read_sql_query("""
SELECT Title, CommentCount, ViewCount, CommentsTotalScore, DisplayName, Reputation, Location FROM (
SELECT Posts.OwnerUserId, Posts.Title, Posts.CommentCount, Posts.ViewCount, CmtTotScr.CommentsTotalScore
FROM (
SELECT PostId, SUM(Score) AS CommentsTotalScore FROM Comments
GROUP BY PostId
) AS CmtTotScr
JOIN Posts ON Posts.Id = CmtTotScr.PostId WHERE Posts.PostTypeId=1
) AS PostsBestComments
JOIN Users ON PostsBestComments.OwnerUserId = Users.Id ORDER BY CommentsTotalScore DESC
LIMIT 10
""", conn)

#pandas
res = solution_4(Users, Posts, Comments)    

#compare
result_sql.equals(res.reset_index(drop=True))


True

**Query explanation:**

- The SQL query asked to display `posts having comments with most total score`. Alomg with User's displayed name, reputation and location). Also the post title, number of comments and the total view count of the post. 

- I got the `sum` of the "Score column" from the `Comments` csv file. After that I renamed it as `CommentsTotalScore`, and then I joined it with `PostId` from the `Comments` csv file, with the filter of `PostTypeId = 1`.
After that I selected the required columns and again did the `inner join`.
Then filtered the data of`CommentTotalScore` in `decreasing order` with the limit of `10`.  
At last I selected the required columns for the output.

## `Fifth Query`

In [16]:
#SQL
result_sql = pd.read_sql_query("""
SELECT Posts.Title, STRFTIME('%Y-%m-%d', Posts.CreationDate) AS Date, VotesByAge.* FROM Posts
JOIN (
SELECT PostId,
MAX(CASE WHEN VoteDate = 'before' THEN Total ELSE 0 END) BeforeCOVIDVotes, MAX(CASE WHEN VoteDate = 'during' THEN Total ELSE 0 END) DuringCOVIDVotes, MAX(CASE WHEN VoteDate = 'after' THEN Total ELSE 0 END) AfterCOVIDVotes, SUM(Total) AS Votes
FROM (
SELECT PostId,
CASE STRFTIME('%Y', CreationDate) WHEN '2022' THEN 'after'
WHEN '2021' THEN 'during' WHEN '2020' THEN 'during' WHEN '2019' THEN 'during' ELSE 'before'
END VoteDate, COUNT(*) AS Total FROM Votes
WHERE VoteTypeId IN (3, 4, 12) GROUP BY PostId, VoteDate
) AS VotesDates
GROUP BY VotesDates.PostId
) AS VotesByAge ON Posts.Id = VotesByAge.PostId WHERE Title NOT IN ('') AND DuringCOVIDVotes > 0 ORDER BY DuringCOVIDVotes DESC, Votes DESC
LIMIT 20
""", conn)

#pandas
res = solution_5(Votes,Posts)   

#compare
result_sql.equals(res)

True

**Query explanation:**

- The SQL query asked to display posts having `maximum downvotes, spam or offensive votes`. By selecting the apporopriate columns, for the pandemic situvation which was divided into three time periods `before, during, after`.

- Firstly I got the required filter for column `VoteTypeId`, afterwards got the desired date format for column `CreationDate`. Then I did the grouping and `dictionary` for  both `WHEN THEN` parts. 

- Then I calculated the sum of column `Total` as `Votes`, afterwards got the desired date format for column `CreationDate` for `Posts` csv file.
Did the filtering for `Title and BeforeCOVIDVotes` as asked.
Finished the `ordering and limiting part`selected the required column for output.


In [ ]:
# after finishing work, we close the connection #
conn.close()